### Projeto
Usando o [dataset do projeto](https://drive.google.com/file/d/17fyteuN2MdGdbP5_Xq_sySN_yH91vTup/view?usp=sharing), crie modelos usando Árvore de Decisão e KNN para identificar se uma pessoa será adimplente ou inadimplente, realizando os pré-processamentos necessários para cada um. Utilize a metodologia de avaliação de sua preferência, mas seu modelo será avaliado em um conjunto apartado. Que conclusões você consegue tirar a partir do modelo?

- Escolham apenas 5 variáveis dentro das 100+ disponíveis

**Grupo de trabalho:**  <br/>
- Adriana Roberta Miceli de Souza <br/>
- Debora Kassem Buturi <br/>
- Helen Cristina de Acypreste Rocha <br/> 
- Marcus Fontes <br/>
- Richard Raphael Banak <br/>


 

>  **_Import de Libs_**

In [68]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_colwidth', 500)

In [147]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, plot_tree
from sklearn.metrics import roc_auc_score
from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

>  **_Leitura dos dados originais_**

In [176]:
# Carregando o CSV
df = pd.read_csv('./projeto_ml2/application_train.csv')

df.dropna()

df = df[(df.TARGET.isnull()==False) & (df.CNT_CHILDREN.isnull()==False) & (df.DAYS_EMPLOYED.isnull()==False) & (df.EXT_SOURCE_1.isnull()==False) & (df.AMT_REQ_CREDIT_BUREAU_YEAR.isnull()==False)]

#df.isnull().sum()

# Mostrar as primeiras linhas
df.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
1,134978,0,Cash loans,F,N,N,0,90000.0,375322.5,14422.5,...,0,0,0,0,0.0,0.0,0.0,1.0,0.0,3.0
4,260639,0,Cash loans,F,N,Y,0,144000.0,675000.0,21906.0,...,0,0,0,0,0.0,0.0,0.0,10.0,0.0,0.0
5,270576,0,Cash loans,M,Y,Y,0,157500.0,630000.0,30307.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
8,212537,0,Revolving loans,F,N,Y,0,162000.0,202500.0,10125.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
10,376566,0,Revolving loans,M,Y,N,0,180000.0,180000.0,9000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,2.0,2.0


In [180]:
# Avaliando a variável resposta:
df['TARGET'].value_counts() , df['TARGET'].value_counts(normalize=True)
# Dados desbalanceados (Default próximo de 8%)

(0    87272
 1     6820
 Name: TARGET, dtype: int64,
 0    0.927518
 1    0.072482
 Name: TARGET, dtype: float64)

In [170]:
# Carregando o CSV
desc = pd.read_csv('./projeto_ml2/HomeCredit_columns_description.csv', encoding =' latin1')
# Mostrar as primeiras linhas
desc.head()

,Unnamed: 0,Table,Row,Description,Special
0,1,application_{train|test}.csv,SK_ID_CURR,ID of loan in our sample,NaN
1,2,application_{train|test}.csv,TARGET,"Target variable (1 - client with payment difficulties: he/she had late payment more than X days on at least one of the first Y installments of the loan in our sample, 0 - all other cases)",NaN
2,5,application_{train|test}.csv,NAME_CONTRACT_TYPE,Identification if loan is cash or revolving,NaN
3,6,application_{train|test}.csv,CODE_GENDER,Gender of the client,NaN
4,7,application_{train|test}.csv,FLAG_OWN_CAR,Flag if the client owns a car,NaN


In [57]:
desc_ = desc[desc['Table'] =='application_{train|test}.csv']
desc_[["Row", "Description"]]

,Row,Description
0,SK_ID_CURR,ID of loan in our sample
1,TARGET,"Target variable (1 - client with payment difficulties: he/she had late payment more than X days on at least one of the first Y installments of the loan in our sample, 0 - all other cases)"
2,NAME_CONTRACT_TYPE,Identification if loan is cash or revolving
3,CODE_GENDER,Gender of the client
4,FLAG_OWN_CAR,Flag if the client owns a car
5,FLAG_OWN_REALTY,Flag if client owns a house or flat
6,CNT_CHILDREN,Number of children the client has
7,AMT_INCOME_TOTAL,Income of the client
8,AMT_CREDIT,Credit amount of the loan
9,AMT_ANNUITY,Loan annuity


>  **_Entendimento inicial do dataset application_test_student_**

In [171]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 94092 entries, 1 to 246002
Columns: 122 entries, SK_ID_CURR to AMT_REQ_CREDIT_BUREAU_YEAR
dtypes: float64(65), int64(41), object(16)
memory usage: 88.3+ MB


In [172]:
df.describe()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
count,94092.000000,94092.000000,94092.000000,9.409200e+04,9.409200e+04,94086.000000,9.400800e+04,94092.000000,94092.000000,94092.00000,...,94092.000000,94092.000000,94092.000000,94092.000000,94092.000000,94092.000000,94092.000000,94092.000000,94092.000000,94092.000000
mean,278054.982708,0.072482,0.515134,1.797213e+05,6.234716e+05,28163.309547,5.621992e+05,0.021098,-14812.962005,33023.17773,...,0.009544,0.000648,0.000808,0.000266,0.006940,0.007408,0.037272,0.287825,0.264411,1.856268
std,102802.224358,0.259286,0.769551,3.956114e+05,4.168799e+05,14847.240534,3.835670e+05,0.014028,3901.553256,108323.36722,...,0.097226,0.025454,0.028409,0.016298,0.087262,0.113206,0.209769,0.921157,0.604746,1.825339
min,100002.000000,0.000000,0.000000,2.700000e+04,4.500000e+04,1980.000000,4.500000e+04,0.000290,-25201.000000,-17170.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,189153.250000,0.000000,0.000000,1.125000e+05,2.842560e+05,17149.500000,2.475000e+05,0.010032,-17526.000000,-2876.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,277893.000000,0.000000,0.000000,1.575000e+05,5.265000e+05,26064.000000,4.545000e+05,0.018850,-14271.500000,-1411.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,367496.500000,0.000000,1.000000,2.250000e+05,8.356050e+05,36090.000000,7.110000e+05,0.028663,-11675.000000,-527.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000
max,456255.000000,1.000000,19.000000,1.170000e+08,4.050000e+06,258025.500000,4.050000e+06,0.072508,-7680.000000,365243.00000,...,1.000000,1.000000,1.000000,1.000000,4.000000,9.000000,8.000000,27.000000,8.000000,18.000000


In [202]:
# Forma mais simples de resolver: Escolha de 5 campos que não precisam de tratamentos (numéricos) para ajuste de uma árvore.

df_treino, df_teste = train_test_split(df, test_size = 0.3, random_state = 10)

var_expl = ['AMT_INCOME_TOTAL', 'CNT_CHILDREN', 'DAYS_EMPLOYED', 'EXT_SOURCE_1','AMT_REQ_CREDIT_BUREAU_YEAR']
var_resp = ['TARGET']

x_treino = df_treino[var_expl]
x_teste = df_teste[var_expl]

# Instanciando e aplicando a normalização:
scaler = StandardScaler()
x_treino_std = scaler.fit_transform(x_treino)
x_teste_std = scaler.transform(x_teste)

y_treino = df_treino[var_resp]
y_teste = df_teste[var_resp]

# Instanciando e aplicando modelos:
regressao_log = LogisticRegression()
regressao_log.fit(x_treino_std,  y_treino['TARGET'])

knn = KNeighborsClassifier(1)
knn.fit(x_treino_std,  y_treino['TARGET'])

decision_tree = DecisionTreeClassifier(max_leaf_nodes = 3, random_state = 50)
decision_tree.fit(x_treino_std,  y_treino['TARGET'])

# Fazendo predições com cada modelo
y_teste_pred_regressao_log = regressao_log.predict(x_teste_std)
y_teste_pred_knn           = knn.predict(x_teste_std)
y_teste_pred_decision_tree = decision_tree.predict(x_teste_std)

#Resultados
accuracy_score(y_teste, y_teste_pred_regressao_log) , accuracy_score(y_teste, y_teste_pred_knn), accuracy_score(y_teste, y_teste_pred_decision_tree)

(0.9275187756837183, 0.8679325492418875, 0.9275187756837183)

In [204]:
#y_treino['TARGET']
y_treino

,TARGET
110169,0
131705,1
152377,0
14090,0
176855,0
...,...
26639,0
24562,0
244568,0
131684,0


In [193]:
print("\nReg-Log\n",classification_report(y_teste, y_teste_pred_regressao_log) ,
      "\nKnn\n", classification_report(y_teste, y_teste_pred_knn)              ,
      "\ndecision_tree\n", classification_report(y_teste, y_teste_pred_knn) )


Reg-Log
               precision    recall  f1-score   support

           0       0.93      1.00      0.96     43626
           1       0.00      0.00      0.00      3420

    accuracy                           0.93     47046
   macro avg       0.46      0.50      0.48     47046
weighted avg       0.86      0.93      0.89     47046
 
Knn
               precision    recall  f1-score   support

           0       0.93      0.93      0.93     43626
           1       0.10      0.10      0.10      3420

    accuracy                           0.87     47046
   macro avg       0.52      0.52      0.52     47046
weighted avg       0.87      0.87      0.87     47046
 
decision_tree
               precision    recall  f1-score   support

           0       0.93      0.93      0.93     43626
           1       0.10      0.10      0.10      3420

    accuracy                           0.87     47046
   macro avg       0.52      0.52      0.52     47046
weighted avg       0.87      0.87      0.8

C:\Users\MARCUSF\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\MARCUSF\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\MARCUSF\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
